# Particionado por fecha de nacimiento para probar el particionamiento por fecha como string

In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
conf = (SparkConf().set("spark.executor.memory", "12g")
                    .set("spark.executor.cores", "5")
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))

spark = SparkSession.builder.master("spark://spark-master:7077").appName("Todo:1.600M").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/01 15:59:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import substring

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True)

df = spark.read.csv("./datos/fakePeopleParallel.csv", sep=";", header = True, schema = esquema)
print(df.count())
df = df.withColumn("yearMon", substring("fecNac",1,7))
df.write.mode("append").partitionBy("yearMon").parquet("./datos/fecNac/fakePeople1600M.parquet")

1600000000


In [3]:
spark.sparkContext.stop()